In [1]:
import h2o
import pandas as pd
import numpy as np

In [37]:
# from sklearn.preprocessing import LabelEncoder
# #Loading data
# df_train = pd.read_csv('../data/raw/train_users.csv')
# df_test = pd.read_csv('../data/raw/test_users.csv')
# labels = df_train['country_destination'].values
# df_train = df_train.drop(['country_destination'], axis=1)
# id_test = df_test['id']
# piv_train = df_train.shape[0]

# #Creating a DataFrame with train+test data
# df_all = pd.concat((df_train, df_test), axis=0, ignore_index=True)
# #Removing id and date_first_booking
# df_all = df_all.drop(['id', 'date_first_booking'], axis=1)
# #Filling nan values
# df_all.loc[df_all.age > 500, 'age'] = 2015 - df_all.age
# df_all = df_all.fillna(-1)


# #####Feature engineering#######
# #date_account_created
# dac = np.vstack(df_all.date_account_created.astype(str).apply(
# lambda x: list(map(int, x.split('-')))).values)
# df_all['dac_year'] = dac[:,0]
# df_all['dac_month'] = dac[:,1]
# df_all['dac_day'] = dac[:,2]
# df_all = df_all.drop(['date_account_created'], axis=1)

# #timestamp_first_active
# tfa = np.vstack(df_all.timestamp_first_active.astype(str).apply(
# lambda x: list(map(int, [x[:4],x[4:6],x[6:8],x[8:10],x[10:12],x[12:14]]))).values)
# df_all['tfa_year'] = tfa[:,0]
# df_all['tfa_month'] = tfa[:,1]
# df_all['tfa_day'] = tfa[:,2]
# df_all = df_all.drop(['timestamp_first_active'], axis=1)

# #Age
# av = df_all.age.values
# df_all['age'] = np.where(np.logical_or(av<14, av>100), -1, av)

# #One-hot-encoding features
# ohe_feats = ['gender', 'signup_method', 'signup_flow', 'language',
# 'affiliate_channel', 'affiliate_provider', 'first_affiliate_tracked',
# 'signup_app', 'first_device_type', 'first_browser']
# for f in ohe_feats:
#     df_all_dummy = pd.get_dummies(df_all[f], prefix=f)
#     df_all = df_all.drop([f], axis=1)
#     df_all = pd.concat((df_all, df_all_dummy), axis=1)

# #Splitting train and test
# vals = df_all.values
# X = vals[:piv_train]
# le = LabelEncoder()
# y = labels
# X_test = vals[piv_train:]

# X = pd.DataFrame(X)
# X['y'] = pd.DataFrame(y)

# X.to_csv('t.csv')

# X_test = pd.DataFrame(X_test)
# X_test['id'] = pd.DataFrame(id_test)
# X_test.to_csv('t2.csv')

In [2]:
h2o.init()
h2o.remove_all() 



No instance found at ip and port: localhost:54321. Trying to start local jar...


JVM stdout: /tmp/tmpUJcSmY/h2o_david_started_from_python.out
JVM stderr: /tmp/tmplSzycs/h2o_david_started_from_python.err
Using ice_root: /tmp/tmp_temhx


Java Version: java version "1.7.0_91"
OpenJDK Runtime Environment (IcedTea 2.6.3) (7u91-2.6.3-0ubuntu0.14.04.1)
OpenJDK 64-Bit Server VM (build 24.91-b01, mixed mode)


Starting H2O JVM and connecting: ............ Connection successful!


H2O cluster uptime:,1 seconds 345 milliseconds
H2O cluster version:,3.6.0.3
H2O cluster name:,H2O_started_from_python
H2O cluster total nodes:,1
H2O cluster total memory:,1.72 GB
H2O cluster total cores:,4
H2O cluster allowed cores:,4
H2O cluster healthy:,True
H2O Connection ip:,127.0.0.1
H2O Connection port:,54321


In [ ]:
df = h2o.import_file("../data/raw/train_users.csv")

In [ ]:
df = df.drop('id')

In [40]:
df = h2o.import_file("../data/raw/train_users.csv")


Parse Progress: [##################################################] 100%


In [45]:
X = df.col_names[:-1]
del X[2]

y = df.col_names[-1]

In [ ]:
# from h2o.grid.grid_search import H2OGridSearch

# ntrees_opt = 1
# max_depth_opt = [2]
# learn_rate_opt = [0.1,0.2]
# hyper_parameters = {"ntrees": ntrees_opt, "max_depth":max_depth_opt, "learn_rate":learn_rate_opt, "distribution":"multinomial"}

# gs = H2OGridSearch(H2OGradientBoostingEstimator, hyper_params=hyper_parameters, )

# gs.train(X, y, training_frame=df)
# print "R2:", dl.r2()

### Deep Learning Model

In [46]:
from h2o.estimators.gbm import H2OGradientBoostingEstimator

dl = H2OGradientBoostingEstimator(ntrees=43,
                                  max_depth=6, 
                                  distribution='multinomial',
                                  stopping_metric="logloss",
                                  learn_rate=0.25,
                                  sample_rate=0.6,
                                  balance_classes=True, 
                                  nfolds=0)

In [47]:
dl.train(X, y, training_frame=df)


gbm Model Build Progress: [##################################################] 100%


In [48]:
print "R2:", dl.r2()

R2: 0.946753982205


The best model:

dl = H2OGradientBoostingEstimator(ntrees=43,
                                  max_depth=6, 
                                  distribution='multinomial',
                                  stopping_metric="logloss",
                                  learn_rate=0.25,
                                  sample_rate=0.6,
                                  balance_classes=True, 
                                  nfolds=0)

### Predictions

In [64]:
test = h2o.import_file("../data/raw/test_users.csv")


Parse Progress: [##################################################] 100%


In [71]:
pred = dl.predict(test.drop('id'))

### Make CSV for submission

In [72]:
p_df = pred.as_data_frame()
p_df = pd.DataFrame(p_df)
p_df = p_df.transpose()
p_df.columns = p_df.iloc[0]
p_df = p_df.reindex(p_df.index.drop(0))
p_df.drop('predict', axis=1, inplace=True)

In [73]:
ids = pd.DataFrame(test.as_data_frame())
ids = ids.transpose()
ids.columns = ids.iloc[0]
ids = ids.reindex(ids.index.drop(0))

In [75]:
A = []
for item in xrange(p_df.shape[0]):
    s = p_df.iloc[item]
    s = s.astype(float)
    s = s.sort_values(ascending=False)
    for i in xrange(5):
        user_id = ids.iloc[item]['id']
        dest = list(s.index[0:5])[i]
        A.append([user_id, dest])

In [76]:
sub = pd.DataFrame(A, columns = ['id', 'country'])

Write CSV file:

In [77]:
sub.to_csv('../data/submissions/sub_h2o.csv',index=False)

In [ ]:
h2o.shutdown()